In [ ]:
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import seaborn as sns

# Overview of the data

### Campaign Desc
* Description (TypeA, TypeB, TypeC)
* Campaign Id
* Start and End Day

### Campaign Table
* Description
* Campaign Id
* Houshold key

### Casual Data
* Product Id
* Store Id
* Week
* Display location (Ad in in-store display? )
* Mailer Location (Featured as ad in weekly mailer)

### Coupon Redemption
* Household Id
* Day
* Coupon Id
* Campaign Id

### Coupon
* Coupon Id
* Product Id
* Campaign Id

### Demographic
* Age (19-65+)
* Marital Status Code (A: Married, B: Single, U: Unknown)
* Income
* Homeowner (Homeowner, Renter, etc.)
* Household composition (Female/Male single, adults with/without kids, etc.)
* Household size
* Number of kids (1-3+)
* Household Id

### Products
* Product Id
* Manufacturer Id
* Departement (grocery, pastry, etc.)
* Brand (national/private)
* Commodity description
* Sub commodity description
* Size

### Transaction
* Household Id
* Manufacturer Id
* Week
* Day
* Time of Day
* Product Id
* Quantity
* Sales value (What shop actually gets)
* Store Id
* Retail discount (Loyalty program of retailer)
* Coupon discount (Shop does not get price before discount)
* Coupon match discount (Shop does get price before discount)






# Loading the data

Let's start by loading the data in pandas dataframes.

## Campaign desc

In [ ]:
df_campaign_desc = pd.read_csv('dunnhumby/campaign_desc.csv')

In [ ]:
df_campaign_desc.head()

In [ ]:
print('Number of campaign: ', len(df_campaign_desc.CAMPAIGN))
print('Unique description: ', df_campaign_desc.DESCRIPTION.unique())

Let's add a column with the duration of each campaign

In [ ]:
df_campaign_desc['DURATION'] = df_campaign_desc['END_DAY'] - df_campaign_desc['START_DAY'] 
df_campaign_desc.sort_values(by = ['DESCRIPTION', 'DURATION'])

## Campaign table

In [ ]:
df_campaign_table = pd.read_csv('dunnhumby/campaign_table.csv')

In [ ]:
df_campaign_table.head()

Let's study the distribution of the number of campaigns each household has benefited from.

In [ ]:
nbins = df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().max()
df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().hist(bins = nbins)
plt.title('Distribution of the number of campaigns per household')
plt.xlabel('Number of campaigns')
plt.ylabel('Number of households')
plt.show()

In [ ]:
print('Mean number of campaigns:', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().mean())
print('Median number of campaigns:', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().median())
print('Max number of campaigns:', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().max())
print('25% of the households have benefited from', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().quantile(q = 0.25),
      'campaigns or less.')
print('75% of the households have benefited from', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().quantile(q = 0.75),
      'campaigns or less.')
print('95% of the households have benefited from', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().quantile(q = 0.95),
      'campaigns or less.')

## Causal data

In [ ]:
df_causal = pd.read_csv('dunnhumby/causal_data.csv')

In [ ]:
df_causal.head()

Let's define two dict for the code meaning of display and mailer.

In [ ]:
display_dict = {0:'Not on Display', 1:'Store Front', 2:'Store Rear', 3:'Front End Cap',
                4:'Mid-Aisle End Cap', 5:'Read End Cap' ,6:'Side-Aisle End Cap', 7:'In-Aisle' ,
                9:'Secondary Location Display' ,'A':'In-Shelf' }
mailer_dict = {0:'Not on ad', 'A':'Interior page feature', 'C':'Interior page line item', 
              'D':'Front page feature', 'F':'Back page feature', 'H':'Wrap front feature',
              'J':'Wrap interior coupon', 'L':'Wrap back feature', 'P':'Interior page coupon',
              'X':'Free on interior page', 'Z':'Free on front page, back page or wrap'}

## Coupon redemption

In [ ]:
df_coupon_redempt = pd.read_csv('dunnhumby/coupon_redempt.csv')

In [ ]:
df_coupon_redempt.head()

## Coupon

In [ ]:
df_coupon = pd.read_csv('dunnhumby/coupon.csv')

In [ ]:
df_coupon.head()

Let's study how many coupons were distributed for each campaign.

In [ ]:
df_coupon.groupby(by = 'CAMPAIGN').count().plot.bar(y = 'COUPON_UPC')
plt.xlabel('Campaign')
plt.ylabel('Number of coupons')
plt.title('Number of coupons per campaign')
plt.show()

We notice there are definetely some campaigns with way more coupons than others. Is this related to the type of campaign?

## Demographics

In [ ]:
df_demo = pd.read_csv('dunnhumby/hh_demographic.csv')

In [ ]:
print(df_demo.head(20))
df_demo["AGE_DESC"].unique()

## Products

In [ ]:
df_products = pd.read_csv('dunnhumby/product.csv')

In [89]:
list_dep = np.sort(df_products["DEPARTMENT"].unique())
list_commodity = np.sort(df_products["COMMODITY_DESC"].unique())
list_sub = np.sort(df_products["SUB_COMMODITY_DESC"].unique())
#df_products[df_products["COMMODITY_DESC"].str.contains("APPLES")]["DEPARTMENT"].unique()
print(list_dep)

frames = {}
for l in list_dep:    
    frames[l] = df_products[df_products["DEPARTMENT"].apply(lambda x : x == l)]

[' ' 'AUTOMOTIVE' 'CHARITABLE CONT' 'CHEF SHOPPE' 'CNTRL/STORE SUP'
 'COSMETICS' 'COUP/STR & MFG' 'DAIRY DELI' 'DELI' 'DELI/SNACK BAR'
 'DRUG GM' 'ELECT &PLUMBING' 'FLORAL' 'FROZEN GROCERY' 'GARDEN CENTER'
 'GM MERCH EXP' 'GRO BAKERY' 'GROCERY' 'HBC' 'HOUSEWARES' 'KIOSK-GAS'
 'MEAT' 'MEAT-PCKGD' 'MEAT-WHSE' 'MISC SALES TRAN' 'MISC. TRANS.'
 'NUTRITION' 'PASTRY' 'PHARMACY SUPPLY' 'PHOTO' 'PORK' 'POSTAL CENTER'
 'PROD-WHS SALES' 'PRODUCE' 'RESTAURANT' 'RX' 'SALAD BAR' 'SEAFOOD'
 'SEAFOOD-PCKGD' 'SPIRITS' 'TOYS' 'TRAVEL & LEISUR' 'VIDEO' 'VIDEO RENTAL']


'for l in list_dep:\n    print(l, ":" ,frames[l].shape[0])'

In [93]:
df_products.groupby("DEPARTMENT").count()["PRODUCT_ID"].sort_values(ascending=False)

DEPARTMENT
GROCERY            39021
DRUG GM            31529
PRODUCE             3118
COSMETICS           3011
NUTRITION           2914
MEAT                2544
MEAT-PCKGD          2427
DELI                2354
PASTRY              2149
FLORAL               938
SEAFOOD-PCKGD        563
MISC. TRANS.         490
SPIRITS              377
SEAFOOD              369
GARDEN CENTER        128
RESTAURANT           102
MISC SALES TRAN       88
SALAD BAR             48
COUP/STR & MFG        39
TRAVEL & LEISUR       28
FROZEN GROCERY        23
KIOSK-GAS             16
                      15
CHEF SHOPPE           14
RX                     9
CNTRL/STORE SUP        4
POSTAL CENTER          3
DAIRY DELI             3
TOYS                   3
VIDEO RENTAL           3
GM MERCH EXP           3
PHOTO                  2
DELI/SNACK BAR         2
PROD-WHS SALES         2
GRO BAKERY             2
CHARITABLE CONT        2
AUTOMOTIVE             2
VIDEO                  2
PORK                   1
ELECT &PLUMBIN

## Transcation data

In [ ]:
df_transaction = pd.read_csv('dunnhumby/transaction_data.csv')

In [ ]:
df_transaction.head()